In [95]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict
from textblob import TextBlob

In [96]:
songs = pd.read_csv('songdata (1).csv')

In [97]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [98]:
songs.isna().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [99]:
songs['text'] = songs['text'].replace(r'\n','')


In [100]:
str=''
for i in range(0,len(songs)):
    str=str+songs['artist'][i]+" "

In [101]:
songs=songs.drop('link', axis=1).reset_index(drop=True)

In [102]:
artists=str.split()

In [103]:
unique_artists=set(artists)

In [104]:
len(unique_artists)

989

In [105]:
def sentiment_analysis(input_song_lyrics, recommended_songs):
    input_sentiment = TextBlob(input_song_lyrics).sentiment.polarity
    sentiment_scores = {}
    
    for song_data in recommended_songs:
        song_lyrics = song_data[1]  # Assuming lyrics are in the second position
        sentiment_score = TextBlob(song_lyrics).sentiment.polarity
        sentiment_scores[song_data[1]] = abs(sentiment_score - input_sentiment),song_data[2]
    
    # Sort the songs based on sentiment difference
    sorted_songs = sorted(sentiment_scores.items(), key=lambda x: x[1])

    # Return top rec_num sentimentally similar songs
    return sorted_songs

In [106]:
def get_selection(artist,song,text,rec_num):
    
    global songs
    
    # add the song for recommendations to the dataset
    x={'artist':artist,'song':song,'text':text}
    x=pd.DataFrame([x])
    songs = songs.sample(n=3000)
    songs=pd.concat([songs,x])
    
    # finding the tfidf scores for all 
    tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
    lyrics_matrix = tfidf.fit_transform(songs['text'])
    
    # finding the cosine similarities for the songs based on tf-idf scores
    cosine_similarities = cosine_similarity(lyrics_matrix) 
    similarities = {}
    
    # stores the top 50 cosime similarities in similarities dictionary 
    for i in range(len(cosine_similarities)):
        similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
        similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'].iloc[x], songs['artist'].iloc[x]) for x in similar_indices][1:]
    
    rec_list=sentiment_analysis(text,similarities[song][1:])
    return rec_list[:rec_num]